In [6]:
# Cell 1: Import and setup
from azureml.core import Workspace, Dataset, Datastore
import os

# Connect to workspace
ws = Workspace.from_config()
print(f"Workspace: {ws.name}")

Workspace: cropdetectionworkspace1994


In [2]:
from azureml.core import Workspace, Dataset, Datastore

ws = Workspace.from_config()
default_ds = ws.get_default_datastore()

# Use the CORRECT path that actually has data
correct_path = 'UI/2025-11-02_161704_UTC/'  

print(f"🎯 Registering dataset with correct path: '{correct_path}'")

# Delete the old dataset if it exists
try:
    old_dataset = Dataset.get_by_name(ws, name='plant_disease_data')
    old_dataset.unregister_all_versions()
    print("🗑️ Deleted old dataset with wrong path")
except:
    print("No old dataset to delete")

# Register with the correct path
dataset = Dataset.File.from_files(path=(default_ds, correct_path))
registered_dataset = dataset.register(
    workspace=ws,
    name='plant_disease_data',
    description='Plant disease dataset - CORRECT PATH',
    create_new_version=True
)

print(f"✅ Dataset registered: '{registered_dataset.name}'")
print(f"📁 Correct path: '{correct_path}'")

🎯 Registering dataset with correct path: 'UI/2025-11-02_161704_UTC/'
🗑️ Deleted old dataset with wrong path
✅ Dataset registered: 'plant_disease_data'
📁 Correct path: 'UI/2025-11-02_161704_UTC/'


In [3]:
# Now test with the correct path
try:
    dataset = Dataset.get_by_name(ws, name='plant_disease_data')
    print(f"✅ Dataset loaded: {dataset.name}")
    
    # Quick test - just get file paths without mounting
    print("🔍 Getting file paths...")
    file_paths = dataset.to_path()
    print(f"📁 Found {len(file_paths)} files")
    print("Sample files:")
    for path in file_paths[:5]:
        print(f"  📄 {path}")
        
except Exception as e:
    print(f"❌ Error: {e}")

✅ Dataset loaded: plant_disease_data
🔍 Getting file paths...
Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=7836a42a-3cc7-472c-84b9-ac0efbdbfcbd) and endpoint type based on configuration
{'infer_column_types': 'False', 'activity': 'to_path'}
{'infer_column_types': 'False', 'activity': 'to_path', 'activityApp': 'FileDataset'}
📁 Found 1532 files
Sample files:
  📄 /lala/Test/Test/Healthy/8ddaa5a5caa5caa8.jpg
  📄 /lala/Test/Test/Healthy/8ddaac1bd6c8cd0a.jpg
  📄 /lala/Test/Test/Healthy/8ddd5ec1c0de38c4.jpg
  📄 /lala/Test/Test/Healthy/8def3f60308ab41b.jpg
  📄 /lala/Test/Test/Healthy/8def4d91382175c3.jpg


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [4]:
# Let's understand your actual folder structure
print("📂 Your actual folder structure based on the files:")
file_paths = dataset.to_path()

# Analyze the folder structure from file paths
folders = set()
for path in file_paths[:20]:  # Check first 20 files
    parts = path.split('/')
    if len(parts) >= 4:
        folder_structure = '/'.join(parts[:4])  # Get first 4 parts
        folders.add(folder_structure)
        print(f"  {path}")

print("\n🎯 Your data structure:")
for folder in sorted(folders):
    print(f"  📁 {folder}/")



📂 Your actual folder structure based on the files:
{'infer_column_types': 'False', 'activity': 'to_path'}
{'infer_column_types': 'False', 'activity': 'to_path', 'activityApp': 'FileDataset'}
  /lala/Test/Test/Healthy/8ddaa5a5caa5caa8.jpg
  /lala/Test/Test/Healthy/8ddaac1bd6c8cd0a.jpg
  /lala/Test/Test/Healthy/8ddd5ec1c0de38c4.jpg
  /lala/Test/Test/Healthy/8def3f60308ab41b.jpg
  /lala/Test/Test/Healthy/8def4d91382175c3.jpg
  /lala/Test/Test/Healthy/8df452e2e38c0b6e.jpg
  /lala/Test/Test/Healthy/8dfae9d78cc32089.jpg
  /lala/Test/Test/Healthy/8e3dbccdfe08c850.jpg
  /lala/Test/Test/Healthy/8e68163c62dc57d5.jpg
  /lala/Test/Test/Healthy/8e6a823cce9ff40c.jpg
  /lala/Test/Test/Healthy/8e77857194a59a87.jpg
  /lala/Test/Test/Healthy/8e79802b3fb770c8.jpg
  /lala/Test/Test/Healthy/8e7986d1ecd36445.jpg
  /lala/Test/Test/Healthy/8e7e70a6878c1c79.jpg
  /lala/Test/Test/Healthy/8e82b1a51bd11afe.jpg
  /lala/Test/Test/Healthy/8e8470687be37378.jpg
  /lala/Test/Test/Healthy/8e858c8397706b7b.jpg
  /lala/Te

In [5]:
%pip install tensorflow==2.12.0 matplotlib
print("✅ Packages installed - RESTART KERNEL NOW!")


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Packages installed - RESTART KERNEL NOW!


In [1]:
from azureml.core import Workspace, Dataset, Run
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

print("TensorFlow version:", tf.__version__)

2025-11-03 16:12:52.133030: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 16:12:52.604883: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 16:12:54.507147: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-03 16:12:54.512778: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-03 16:12:59.008689: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

TensorFlow version: 2.12.0


In [2]:
# Load your dataset
ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='plant_disease_data')
print("✅ Dataset loaded:", dataset.name)

# Get all file paths
file_paths = dataset.to_path()
print(f"📁 Total files found: {len(file_paths)}")

# Analyze class distribution
print("\n🔍 Analyzing your data classes:")
class_counts = {}
for path in file_paths:
    # Extract class from path: /lala/Test/Test/Healthy/image.jpg → Healthy
    parts = path.split('/')
    if len(parts) >= 2:
        class_name = parts[-2]  # Second last part is class name
        class_counts[class_name] = class_counts.get(class_name, 0) + 1

for class_name, count in class_counts.items():
    print(f"  {class_name}: {count} images")

print("\n🎯 Your folder structure:")
print("  /lala/Test/Test/Healthy/")
print("  /lala/Test/Test/Powdery/") 
print("  /lala/Test/Test/Rust/")
print("  /lala/Train/Train/Healthy/")
print("  /lala/Validation/Validation/Healthy/")

✅ Dataset loaded: plant_disease_data
Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=7836a42a-3cc7-472c-84b9-ac0efbdbfcbd) and endpoint type based on configuration
{'infer_column_types': 'False', 'activity': 'to_path'}
{'infer_column_types': 'False', 'activity': 'to_path', 'activityApp': 'FileDataset'}
📁 Total files found: 1532

🔍 Analyzing your data classes:
  Healthy: 528 images
  Powdery: 500 images
  Rust: 504 images

🎯 Your folder structure:
  /lala/Test/Test/Healthy/
  /lala/Test/Test/Powdery/
  /lala/Test/Test/Rust/
  /lala/Train/Train/Healthy/
  /lala/Validation/Validation/Healthy/


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()
Timeout was exceeded in force_flush().
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [3]:
# Since mounting doesn't work, we'll download the data temporarily
# This is the reliable approach

print("📥 Downloading dataset for training...")
local_path = dataset.download(target_path='./plant_data', overwrite=True)
print(f"✅ Data downloaded to: {local_path}")

# Now we can work with local files
base_path = './plant_data'
print(f"📂 Local data structure: {os.listdir(base_path)}")

# Your actual data is in the lala folder
data_base_path = os.path.join(base_path, 'lala')
print(f"🎯 Training data path: {data_base_path}")
print(f"📁 Available: {os.listdir(data_base_path)}")

📥 Downloading dataset for training...
{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}
✅ Data downloaded to: ['/mnt/batch/tasks/shared/LS_root/mounts/clusters/divinstance/code/Users/TRISHALA.20221IST0050/plant_data/lala/Test/Test/Healthy/8ddaa5a5caa5caa8.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/divinstance/code/Users/TRISHALA.20221IST0050/plant_data/lala/Test/Test/Healthy/8def4d91382175c3.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/divinstance/code/Users/TRISHALA.20221IST0050/plant_data/lala/Test/Test/Healthy/8e68163c62dc57d5.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/divinstance/code/Users/TRISHALA.20221IST0050/plant_data/lala/Test/Test/Healthy/8e7986d1ecd36445.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/clusters/divinstance/code/Users/TRISHALA.20221IST0050/plant_data/lala/Test/Test/Healthy/8e858c8397706b7b.jpg', '/mnt/batch/tasks/shared/LS_root/mounts/cl

In [4]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout  # Added Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Added LR scheduler

In [5]:

train_path = './plant_data/lala/Train/Train'
val_path = './plant_data/lala/Validation/Validation'


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,            # Rotate images more for variation
    width_shift_range=0.3,        # Slightly stronger horizontal shift
    height_shift_range=0.3,       # Slightly stronger vertical shift
    horizontal_flip=True,         # Flip images horizontally
    zoom_range=0.3,               # Zoom for scale variation
    brightness_range=[0.8, 1.2],  # Change brightness for lighting variation
    shear_range=0.2               # Slight shear for perspective
)
val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale validation


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


def create_model(num_classes=3):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False  # Freeze base layers for initial training
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Convert features to 1D
    x = Dense(256, activation='relu')(x)  # Custom dense layer
    x = Dropout(0.5)(x)  # Dropout to reduce overfitting
    predictions = Dense(num_classes, activation='softmax')(x)  # Output layer
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


model = create_model(num_classes=3)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  # Stop if no improvement
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7, verbose=1)  # Reduce LR
callbacks = [early_stop, lr_schedule]


history = model.fit(
    train_generator,
    epochs=10,                     # Train first 10 epochs on frozen base
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)


for layer in model.layers[-30:]:
    layer.trainable = True  # Unfreeze last 30 layers for fine-tuning

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Smaller LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history_finetune = model.fit(
    train_generator,
    epochs=10,                     # Fine-tune for 10 more epochs
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)


model.save('plant_disease_model.h5')  # Save trained model
print("💾 Model saved as 'plant_disease_model.h5'")


final_accuracy = history_finetune.history['accuracy'][-1]
final_val_accuracy = history_finetune.history['val_accuracy'][-1]
print(f"🎯 Final Training Accuracy: {final_accuracy:.2%}")
print(f"🎯 Final Validation Accuracy: {final_val_accuracy:.2%}")

Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Epoch 1/10
42/42 [==============================] - 338s 8s/step - loss: 1.2925 - accuracy: 0.3169 - val_loss: 1.0921 - val_accuracy: 0.3000 - lr: 0.0010
Epoch 2/10
42/42 [==============================] - 303s 7s/step - loss: 1.1169 - accuracy: 0.3283 - val_loss: 1.0921 - val_accuracy: 0.3333 - lr: 0.0010
Epoch 3/10
42/42 [==============================] - 294s 7s/step - loss: 1.0987 - accuracy: 0.3510 - val_loss: 1.0794 - val_accuracy: 0.4333 - lr: 0.0010
Epoch 4/10
42/42 [==============================] - 325s 8s/step - loss: 1.1009 - accuracy: 0.3487 - val_loss: 1.0820 - val_accuracy: 0.4500 - lr: 0.0010
Epoch 5/10
42/42 [==============================] - ETA: 0s - loss: 1.1011 - accuracy: 0.3442 
Epoch 5: ReduceLRO

2025-11-03 16:20:22.361473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-11-03 16:46:31.903719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-11-03 16:51:56.244165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [2]:
# Save the trained model
model.save('plant_disease_model.h5')
print("💾 Model saved as 'plant_disease_model.h5'")

# Log final accuracy
final_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
print(f"🎯 Final Training Accuracy: {final_accuracy:.2%}")
print(f"🎯 Final Validation Accuracy: {final_val_accuracy:.2%}")

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()